In [1]:
import yaml
import argparse
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
from EnvRunner import GymRunner
from Explanations_Models.DT_LIME.LIME import LIME
from Explanations_Models.DT_LIME.sampling_methods import Uniform_Sampler
import json
from ExplanationRunner import MetricGetter
yaml.add_constructor("!join", join)
config = yaml.load(open("config_envs/cartpole.yml"), Loader= yaml.FullLoader)
config.update(yaml.load(open("config_explanations/TESTER.yml"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights()
MG = MetricGetter(config, Runner)

device:  cpu


In [2]:
thing = MG.sample_rate()



In [3]:
print(thing)

{'3': {'PercentCorrect': [0.88, 0.495, 0.495, 0.51, 0.855], 'EpisodeDistance': [0.09867910295724869, 0.7760151028633118, 0.6228576302528381, 0.8617138266563416, 0.8202632665634155], 'Episode_Length_Distance': [0, 0, 0, 0, 0], 'Depth': [1, 0, 0, 1, 1], 'Breadth': [2, 1, 1, 2, 2], 'TopSplits': ['angle_vel', 'angle', 'angle', 'angle', 'angle_vel'], 'Path': 'SavedTrees/Policy/gini/3/tree_4.pkl'}, '5': {'PercentCorrect': [0.5, 0.84, 0.745, 0.495, 0.51], 'EpisodeDistance': [1.1155929565429688, 0.21740132570266724, 0.24682459235191345, 0.7213442921638489, 0.7014747262001038], 'Episode_Length_Distance': [0, 0, 0, 0, 0], 'Depth': [1, 1, 1, 0, 1], 'Breadth': [2, 2, 2, 1, 2], 'TopSplits': ['angle', 'angle_vel', 'vel', 'angle', 'x'], 'Path': 'SavedTrees/Policy/gini/5/tree_4.pkl'}, '10': {'PercentCorrect': [0.95, 0.78, 0.905, 0.905, 0.755], 'EpisodeDistance': [0.8620252013206482, 0.32922300696372986, 0.2525949478149414, 0.19113433361053467, 0.1255502551794052], 'Episode_Length_Distance': [0, 0, 0, 

In [4]:
print(type(thing))

<class 'dict'>


In [16]:
for i in thing.keys():
    for j in thing[i].keys():
        if type(thing[i][j]) == list:
            print(thing[i][j])
            for k in range(len(thing[i][j])):
                print(type(thing[i][j][k]))

[0.51, 0.495, 0.74, 0.51, 0.9]
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
[0.99973714, 0.77664727, 0.10023394, 0.62364614, 0.12209195]
<class 'numpy.float32'>
<class 'numpy.float32'>
<class 'numpy.float32'>
<class 'numpy.float32'>
<class 'numpy.float32'>
[0, 0, 0, 0, 0]
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
[0, 0, 1, 1, 1]
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
[1, 1, 2, 2, 2]
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
['angle', 'angle', 'vel', 'angle', 'angle_vel']
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
[0.81, 0.805, 0.885, 0.865, 0.875]
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
[0.15928325, 0.06263796, 0.08045942, 0.6393279, 0.2653419]
<class 'numpy.float32'>
<class 'numpy.float32'>
<class 'numpy.float32'>
<class

In [5]:
MG.Saver(thing)

'MetricFiles/CartPole-v1/Policy/gini.json'

In [11]:
thing = LIME(config, Runner)
thing.surr_model.Load(filename= "tree_4.pkl", path="SavedTrees/Policy/gini/3")

In [12]:
thing.surr_model.display_tree()

In [6]:
thing

{'3': {'PercentCorrect': [0.88, 0.495, 0.495, 0.51, 0.855],
  'EpisodeDistance': [0.09867910295724869,
   0.7760151028633118,
   0.6228576302528381,
   0.8617138266563416,
   0.8202632665634155],
  'Episode_Length_Distance': [0, 0, 0, 0, 0],
  'Depth': [1, 0, 0, 1, 1],
  'Breadth': [2, 1, 1, 2, 2],
  'TopSplits': ['angle_vel', 'angle', 'angle', 'angle', 'angle_vel'],
  'Path': 'SavedTrees/Policy/gini/3/tree_4.pkl'},
 '5': {'PercentCorrect': [0.5, 0.84, 0.745, 0.495, 0.51],
  'EpisodeDistance': [1.1155929565429688,
   0.21740132570266724,
   0.24682459235191345,
   0.7213442921638489,
   0.7014747262001038],
  'Episode_Length_Distance': [0, 0, 0, 0, 0],
  'Depth': [1, 1, 1, 0, 1],
  'Breadth': [2, 2, 2, 1, 2],
  'TopSplits': ['angle', 'angle_vel', 'vel', 'angle', 'x'],
  'Path': 'SavedTrees/Policy/gini/5/tree_4.pkl'},
 '10': {'PercentCorrect': [0.95, 0.78, 0.905, 0.905, 0.755],
  'EpisodeDistance': [0.8620252013206482,
   0.32922300696372986,
   0.2525949478149414,
   0.1911343336105346

In [10]:
thing2 = json.load(open('MetricFiles/CartPole-v1/Policy/gini.json', 'r'))
#plot percent_correct
samples = []
means = []
covs = []
for key in thing.keys():
    samples.append(int(key))
    means.append(np.mean(thing2[key]["PercentCorrect"]))
    covs.append()
print(x, y)

[3, 5, 10] [0.647, 0.618, 0.859]


In [11]:
thing2['5']

{'PercentCorrect': [0.5, 0.84, 0.745, 0.495, 0.51],
 'EpisodeDistance': [1.1155929565429688,
  0.21740132570266724,
  0.24682459235191345,
  0.7213442921638489,
  0.7014747262001038],
 'Episode_Length_Distance': [0, 0, 0, 0, 0],
 'Depth': [1, 1, 1, 0, 1],
 'Breadth': [2, 2, 2, 1, 2],
 'TopSplits': ['angle', 'angle_vel', 'vel', 'angle', 'x'],
 'Path': 'SavedTrees/Policy/gini/5/tree_4.pkl'}